In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# === Image Processing Filters and Enhancements ===
def apply_median(img):
    return cv2.medianBlur(img, 5)

def apply_gaussian(img):
    return cv2.GaussianBlur(img, (5, 5), 0)

def apply_bilateral(img):
    return cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)

def enhance_clahe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

def enhance_hist_eq(img):
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    yuv[:, :, 0] = cv2.equalizeHist(yuv[:, :, 0])
    return cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)

def adjust_saturation(img, scale=1.5):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[:, :, 1] *= scale
    hsv[:, :, 1] = np.clip(hsv[:, :, 1], 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

def adjust_hue(img, shift=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv[:, :, 0] = (hsv[:, :, 0] + shift) % 180
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def pencil_sketch_effect(img):
    gray, sketch = cv2.pencilSketch(img, sigma_s=60, sigma_r=0.07, shade_factor=0.05)
    return sketch

def cartoon_effect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray, 7)
    edges = cv2.adaptiveThreshold(blur, 255,
                                   cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 9, 10)
    color = cv2.bilateralFilter(img, 9, 300, 300)
    cartoon = cv2.bitwise_and(color, color, mask=edges)
    return cartoon

def sepia_tone(img):
    kernel = np.array([[0.272, 0.534, 0.131],
                       [0.349, 0.686, 0.168],
                       [0.393, 0.769, 0.189]])
    sepia = cv2.transform(img, kernel)
    return np.clip(sepia, 0, 255).astype(np.uint8)

def quality_metrics(original, processed):
    gray_orig = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    gray_proc = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
    psnr = peak_signal_noise_ratio(gray_orig, gray_proc)
    ssim = structural_similarity(gray_orig, gray_proc)
    return psnr, ssim

# === GUI SETUP ===
class ImageEditorGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Effects GUI")
        self.original_img = None
        self.processed_img = None
        self.tk_img_original = None
        self.tk_img_processed = None

        self.label = tk.Label(root, text="Upload an image")
        self.label.pack()

        self.canvas_frame = tk.Frame(root)
        self.canvas_frame.pack()

        self.canvas_original = tk.Label(self.canvas_frame)
        self.canvas_original.pack(side=tk.LEFT)

        self.canvas_processed = tk.Label(self.canvas_frame)
        self.canvas_processed.pack(side=tk.RIGHT)

        self.btn_upload = tk.Button(root, text="Upload Image", command=self.upload_image)
        self.btn_upload.pack()

        self.effect_menu = ttk.Combobox(root, values=[
            "Median Filter",
            "Gaussian Blur",
            "Bilateral Filter",
            "CLAHE",
            "Histogram Equalization",
            "Saturation",
            "Hue",
            "Pencil Sketch",
            "Cartoon",
            "Sepia"
        ])
        self.effect_menu.set("Select Effect")
        self.effect_menu.pack()

        self.btn_apply = tk.Button(root, text="Apply Effect", command=self.apply_effect)
        self.btn_apply.pack()

        self.result_label = tk.Label(root, text="")
        self.result_label.pack()

    def upload_image(self):
        path = filedialog.askopenfilename()
        if path:
            self.original_img = cv2.imread(path)
            self.processed_img = None
            self.display_images()

    def display_images(self):
        if self.original_img is not None:
            img_rgb = cv2.cvtColor(self.original_img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_resized = img_pil.resize((300, 250))
            self.tk_img_original = ImageTk.PhotoImage(img_resized)
            self.canvas_original.configure(image=self.tk_img_original)
            self.canvas_original.image = self.tk_img_original

        if self.processed_img is not None:
            img_rgb = cv2.cvtColor(self.processed_img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_resized = img_pil.resize((300, 250))
            self.tk_img_processed = ImageTk.PhotoImage(img_resized)
            self.canvas_processed.configure(image=self.tk_img_processed)
            self.canvas_processed.image = self.tk_img_processed

    def apply_effect(self):
        if self.original_img is None:
            return

        choice = self.effect_menu.get()
        img = self.original_img

        if choice == "Median Filter":
            result = apply_median(img)
        elif choice == "Gaussian Blur":
            result = apply_gaussian(img)
        elif choice == "Bilateral Filter":
            result = apply_bilateral(img)
        elif choice == "CLAHE":
            result = enhance_clahe(img)
        elif choice == "Histogram Equalization":
            result = enhance_hist_eq(img)
        elif choice == "Saturation":
            result = adjust_saturation(img)
        elif choice == "Hue":
            result = adjust_hue(img)
        elif choice == "Pencil Sketch":
            result = pencil_sketch_effect(img)
        elif choice == "Cartoon":
            result = cartoon_effect(img)
        elif choice == "Sepia":
            result = sepia_tone(img)
        else:
            return

        self.processed_img = result
        psnr, ssim = quality_metrics(img, result)
        self.result_label.config(text=f"PSNR: {psnr:.2f}, SSIM: {ssim:.4f}")
        self.display_images()

# === RUN ===
if __name__ == '__main__':
    root = tk.Tk()
    app = ImageEditorGUI(root)
    root.mainloop()